In [1]:
import pandas as pd
import collections

import etl_idesp
from util import gera_id_gestao

In [2]:
NOMES_SARESP = [
    'dados/SARESP_escolas_2014.csv',
    'dados/SARESP_escolas_2015.csv',
    'dados/SARESP_escolas_2016.csv',
    'dados/SARESP_escolas_2017.csv',
    'dados/SARESP_escolas_2018.csv']

ANOS_SARESP = [2014, 2015, 2016, 2017, 2018]

def para_numero(num_entrada):
    num_entrada = str(num_entrada)
    num_entrada = num_entrada.replace('.','')
    num_entrada = num_entrada.replace(',','.')
    try:
        num_entrada = float(num_entrada)
        return num_entrada
    except:
        return None
    

In [3]:
dados_saresp = []
for arquivo, ano in zip(NOMES_SARESP, ANOS_SARESP):
    dados = pd.read_csv(arquivo, sep=';', encoding='latin-1')
    dados['ano'] = ano
    dados_saresp.append(dados)
saresp = dados_saresp[0]
for i in range(1,len(dados_saresp)):
    saresp.append(dados_saresp[1])
saresp = saresp.copy()    

In [4]:
saresp.loc[:,'medprof'] = saresp.medprof.apply(para_numero)
saresp = saresp.dropna(subset=['medprof'])
saresp.loc[:,'medprof'] = saresp.medprof.astype(float)

In [5]:
len(saresp)

76511

In [6]:
id_gestao = []
for i in saresp.index:
    id_gestao.append(gera_id_gestao(saresp.loc[i,'CODESC'],saresp.loc[i,'ano']))
saresp['id_gestao'] = id_gestao

In [7]:
sar = {}
sar['id_gestao'] = []
sar['id_escola'] = []
sar['ano_gestao'] = []
sar['nota_saresp'] = []

for nome, dados in saresp.groupby('id_gestao'):
    sar['id_gestao'].append(nome)
    sar['id_escola'].append(dados.CODESC.iloc[0])
    sar['ano_gestao'].append(dados.ano.iloc[0])
    sar['nota_saresp'].append(dados.medprof.median())
sar = pd.DataFrame(sar)

In [8]:
sar.nota_saresp.describe()

count    5039.000000
mean      244.032516
std        25.759206
min       151.500000
25%       227.200000
50%       246.600000
75%       261.900000
max       381.600000
Name: nota_saresp, dtype: float64

In [10]:
sar.head(3)

,id_gestao,id_escola,ano_gestao,nota_saresp
0,esc-10005-ano-2014,10005,2014,247.4
1,esc-10017-ano-2014,10017,2014,268.2
2,esc-10029-ano-2014,10029,2014,246.0


In [1]:
import etl_saresp

In [2]:
sar = etl_saresp.etl()

In [5]:
sar.nota_saresp.isnull().sum()

0